In [12]:
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding



# DATABASE

In [13]:
# Simplified representation of a user database
user_database = {'sharif': {'password': 'password123', 'salt': b'somesalt'}}

# Ticket Genrating

In [24]:
# Simplified representation of the Key Distribution Center (KDC)
def generate_ticket_granting_ticket(username):
    # Generate TGT with session key
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()

    # Serialize public key
    public_key_bytes = public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    print("Generate TGT with session key\n",public_key_bytes,'\n\n\n')

    # Return TGT
    return {
        'username': username,
        'session_key': private_key,
        'public_key': public_key_bytes
    }

In [25]:
def generate_service_ticket(tgt, service):
    # Generate service ticket with a new session key
    service_session_key = Fernet.generate_key()
    
    # Ensure the key size is compatible with AES (128, 192, or 256 bits)
    service_session_key = service_session_key[:32]  # Use the first 32 bytes for a 256-bit key
    
    print("\nService_session_key\n",service_session_key,"\n\n\n") 
    # Use the public key for encryption
    public_key = serialization.load_pem_public_key(tgt['public_key'], backend=default_backend())
    encrypted_service_session_key = public_key.encrypt(
        service_session_key,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    
    # Return service ticket
    return {
        'username': tgt['username'],
        'encrypted_service_session_key': encrypted_service_session_key,
        'service_session_key': service_session_key,
    }

In [26]:
def main():
    # User authentication
    username = input("Enter username: ")
    password = input("Enter password: ")

    # Verify user credentials (this is a simplified example; use secure methods in practice)
    user = user_database.get(username)
    if not user or user['password'] != password:
        print("Invalid credentials")
        return

    # Generate TGT
    tgt = generate_ticket_granting_ticket(username)

    # Request service ticket
    print("Available services: File Server, Email Server")
    service = input("Enter the service you want to access: ")

    # Generate service ticket
    service_ticket = generate_service_ticket(tgt, service)

    # Simulate service server validation (decrypt service ticket)
    decrypted_service_session_key = tgt['session_key'].decrypt(
        service_ticket['encrypted_service_session_key'],
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )

    # Simulate using the decrypted service session key to encrypt and decrypt a message
    sample_message = b"Hello, this is a sample message for encryption and decryption."

    # Pad the sample message to ensure its length is a multiple of the block size
    block_size = 16  # 128 bits for AES
    padded_message = sample_message + b'\0' * (block_size - len(sample_message) % block_size)

    # Encrypt the message using the decrypted service session key
    cipher = Cipher(algorithms.AES(decrypted_service_session_key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(padded_message) + encryptor.finalize()

    # Decrypt the message using the decrypted service session key
    decryptor = cipher.decryptor()
    decrypted_message = decryptor.update(encrypted_message) + decryptor.finalize()

    print(f"Welcome, {username}! You have successfully obtained a service ticket for {service}.\n")
    print(f"Your decrypted service session key for {service}: {decrypted_service_session_key}\n")
    print(f"Sample Message: {sample_message.decode()}\n")
    print(f"Padded Message: {padded_message}\n")
    print(f"Encrypted Message: {encrypted_message}\n")
    print("Decrypted Message:", decrypted_message.decode().rstrip('\0'))

if __name__ == "__main__":
    main()


Enter username: sharif
Enter password: password123
Generate TGT with session key
 b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAmhNxR5XxmPlb7d3y4r3X\nZUDYt6xtqO1UgMqxmjSN/tqa8uMPc2igkAsA0VC7E715o135tlxIE6gjRgKX5Cx4\n45Vru06er9O2QXmCy8LjoCUYAebCkvJ+TABvkyUkkPAA0kpbtiiB8iasBnu3x+NO\n+ARQao2/jTIXpc0si78vH6jt9Vc7x5fShDTeVA1I2mwfGt6pcrKxMMOajtNdjUaL\nIHWljxDqBbIbKALzy7DhbXlIttw768IH3VQyw5UaP/glntZtY270b4IWCm/HY2h5\nRWxVluqeVSzjAss/NCsEXoJpBkBTaobsc53hfkHxBZ+c0gtbk6G5ScdnMGeAlpb3\ngQIDAQAB\n-----END PUBLIC KEY-----\n' 



Available services: File Server, Email Server
Enter the service you want to access: File Server

Service_session_key
 b'pKptp27TdYNyt3vBAt32O_jMv6AKyXbq' 



Welcome, sharif! You have successfully obtained a service ticket for File Server.

Your decrypted service session key for File Server: b'pKptp27TdYNyt3vBAt32O_jMv6AKyXbq'

Sample Message: Hello, this is a sample message for encryption and decryption.

Padded Message: b'Hello, this is a sampl